In [2]:
import numpy as np, pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk.tokenize import sent_tokenize, word_tokenize
en_nlp = spacy.load('en')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import precision_recall_fscore_support
from sklearn.svm import LinearSVC

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data = pd.read_csv("data/train_unsupervised_predicted.csv")
print(data.shape)
data.head()

(82174, 12)


,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,269,in the late 1990s,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.04716371 0.08111103 ... 0.0...,"[0.6082267463207245, 0.7082076072692871, 0.669...","[8.0352955, 9.462915, 6.914433, 9.608288]",0,2
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,207,singing and dancing,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.05823731 0.07162362 ... 0.0...,"[0.5250032842159271, 0.51096311211586, 0.62473...","[6.9849253, 6.9543734, 6.4365134, 8.201017]",1,2
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,526,2003,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,3,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.04716371 0.11232486 ... 0.0...,"[0.4191964268684387, 0.4746186137199402, 0.584...","[5.770602, 6.6236463, 6.2470593, 7.1245747]",0,0
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,166,"Houston, Texas",['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.03403048 0.08177199 ... 0.0...,"[0.5637381374835968, 0.5945537984371185, 0.651...","[7.3106747, 7.8408995, 6.481517, 8.796563]",0,2
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,276,late 1990s,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.06298249 0.10006339 ... 0.0...,"[0.4646044373512268, 0.5985883176326752, 0.589...","[6.2806025, 8.1350765, 6.1614323, 8.060831]",0,2


In [15]:
data.head(5)['question'].tolist()

['When did Beyonce start becoming popular?',
 'What areas did Beyonce compete in when she was growing up?',
 "When did Beyonce leave Destiny's Child and become a solo singer?",
 'In what city and state did Beyonce  grow up? ',
 'In which decade did Beyonce become famous?']

## Baseline Features and Model
- common stemmed non-stop word in sentence and question

In [3]:
ps = PorterStemmer()

In [4]:
stop_words = set(stopwords.words('english')) 

In [5]:
def common_words(x):
    """ Find the number of common stemmed words of each sentence with the question"""
    
    question = x["question"].lower()
    sentences = en_nlp(x["context"].lower()).sents
    
    question_word = word_tokenize(question)
    
    num_common_word = []
    for i, sent in enumerate(sentences):
        sent_word = word_tokenize(str(sent))
        
        num = len(list(set(question_word) & set(sent_word) - stop_words))
        num_common_word.append(num)
        
    diff = 10 - len(num_common_word)
    num_common_word.extend([None] * diff)
            
    return num_common_word

In [6]:
data['num_common_word'] = data.apply(common_words, axis = 1)

In [7]:
def get_columns_from_common_word(train):
    
    columns = ['column_common_0', 'column_common_1', 'column_common_2', 'column_common_3', 'column_common_4',
              'column_common_5', 'column_common_6', 'column_common_7', 'column_common_8', 'column_common_9']
    tmp = pd.DataFrame(columns=columns)
    
    for i in range(len(train)):
        tmp.loc[i,] = train['num_common_word'][i][:10]

    return tmp

In [8]:
baseline_feature = get_columns_from_common_word(data)
baseline_feature.head()

,column_common_0,column_common_1,column_common_2,column_common_3,column_common_4,column_common_5,column_common_6,column_common_7,column_common_8,column_common_9
0,0,0,0,0,None,None,None,None,None,None
1,0,0,0,0,None,None,None,None,None,None
2,1,4,1,2,None,None,None,None,None,None
3,0,0,0,0,None,None,None,None,None,None
4,0,0,0,0,None,None,None,None,None,None


In [9]:
baseline_feature = baseline_feature.fillna(0)

In [4]:
#baseline_feature.to_csv('data/baseline_feature.csv', index=False)
baseline_feature = pd.read_csv('data/baseline_feature.csv')

In [49]:
train_x, test_x, train_y, test_y = train_test_split(baseline_feature, data['target'], train_size=0.7, random_state = 42)
print(train_x.shape)

(57521, 10)


In [50]:
val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, train_size=0.5, random_state = 42)
print(val_x.shape)
print(test_x.shape)

(12326, 10)
(12327, 10)


### Baseline Logistic Model

In [51]:
baseline_logistic = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(train_x, train_y)

In [52]:
y_pred = baseline_logistic.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.6843514237040642, 0.6843514237040642, 0.6843514237040642, None)
Macro Test Accuracy :  (0.5697101669591711, 0.4720041656258141, 0.5054141546364483, None)
accuracy:  0.6843514237040642


## Semantic Similarity Features Added
- cosine similarity & euclidean distance from InferSent encoder

### Feature Engineer

In [3]:
def create_features(data):
    """Use Euclidean distance and Cosine similarity of question with each sentence as features"""
    
    train = pd.DataFrame()
     
    for k in range(len(data["euclidean_dis"])):
        dis = ast.literal_eval(data["euclidean_dis"][k])
        for i in range(len(dis)):
            train.loc[k, "column_euc_"+"%s"%i] = dis[i]
    
    for k in range(len(data["cosine_sim"])):
        dis = ast.literal_eval(data["cosine_sim"][k].replace("nan","1"))
        for i in range(len(dis)):
            train.loc[k, "column_cos_"+"%s"%i] = dis[i]
            
    train["target"] = data["target"]
    return train

In [5]:
data2 = create_features(data)
print(data2.shape)
data2.head()

(82174, 21)


,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,8.035296,9.462915,6.914433,9.608288,NaN,NaN,NaN,NaN,NaN,NaN,...,0.708208,0.669542,0.653088,NaN,NaN,NaN,NaN,NaN,NaN,1
1,6.984925,6.954373,6.436513,8.201017,NaN,NaN,NaN,NaN,NaN,NaN,...,0.510963,0.624731,0.549588,NaN,NaN,NaN,NaN,NaN,NaN,1
2,5.770602,6.623646,6.247059,7.124575,NaN,NaN,NaN,NaN,NaN,NaN,...,0.474619,0.584199,0.461928,NaN,NaN,NaN,NaN,NaN,NaN,3
3,7.310675,7.840899,6.481517,8.796563,NaN,NaN,NaN,NaN,NaN,NaN,...,0.594554,0.651650,0.605872,NaN,NaN,NaN,NaN,NaN,NaN,1
4,6.280602,8.135077,6.161432,8.060831,NaN,NaN,NaN,NaN,NaN,NaN,...,0.598588,0.589348,0.535108,NaN,NaN,NaN,NaN,NaN,NaN,1


### Replace NA with Max Value

In [11]:
# replace NA with max value
data2.describe()

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
count,82174.000000,80551.000000,74733.000000,62163.000000,44035.000000,27871.000000,16205.000000,8739.000000,4203.000000,1568.000000,...,80551.000000,74733.000000,62163.000000,44035.000000,27871.000000,16205.000000,8739.000000,4203.000000,1568.000000,82174.000000
mean,6.288110,6.436538,6.474164,6.507545,6.538072,6.556812,6.620354,6.625505,6.631375,6.588766,...,0.561409,0.567018,0.576791,0.585310,0.591687,0.601115,0.608199,0.616436,0.605842,1.835252
std,1.928739,1.855262,1.830071,1.800666,1.768906,1.721710,1.743979,1.707930,1.753281,1.596566,...,0.148923,0.148926,0.151471,0.150087,0.150561,0.152093,0.155494,0.163133,0.152374,1.752255
min,0.393842,0.345350,0.607994,0.352450,0.411123,0.588479,0.387424,0.986649,0.960692,1.541692,...,0.029885,0.064520,0.034087,0.038803,0.043521,0.035589,0.092596,0.093670,0.138453,-1.000000
25%,4.992334,5.230613,5.284672,5.334724,5.381540,5.441370,5.485276,5.525175,5.470928,5.567340,...,0.463416,0.469886,0.477458,0.487565,0.493391,0.503716,0.508753,0.510071,0.505476,0.000000
50%,6.031953,6.251079,6.287081,6.324439,6.354094,6.384669,6.425293,6.433084,6.423402,6.459106,...,0.552978,0.558806,0.566658,0.574517,0.579970,0.586565,0.590999,0.597116,0.590396,1.000000
75%,7.296540,7.415529,7.450157,7.457891,7.475518,7.470943,7.531432,7.526784,7.548876,7.471178,...,0.649365,0.654462,0.663348,0.669395,0.676214,0.682569,0.688229,0.700769,0.684157,3.000000
max,23.558975,40.290561,33.286404,34.140771,36.115491,35.623532,39.640774,19.253563,18.217110,15.516279,...,1.489619,1.397736,1.490736,1.360776,1.355459,1.415495,1.354246,1.315012,1.159472,9.000000


In [235]:
subset1 = data2.iloc[:,:10].fillna(41)
subset2 = data2.iloc[:,10:-1].fillna(1.5)
sem_similar_feature = pd.concat([subset1, subset2],axis=1, join_axes=[subset1.index])
sem_similar_feature.head()

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,column_cos_0,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9
0,8.035296,9.462915,6.914433,9.608288,41.0,41.0,41.0,41.0,41.0,41.0,0.608227,0.708208,0.669542,0.653088,1.5,1.5,1.5,1.5,1.5,1.5
1,6.984925,6.954373,6.436513,8.201017,41.0,41.0,41.0,41.0,41.0,41.0,0.525003,0.510963,0.624731,0.549588,1.5,1.5,1.5,1.5,1.5,1.5
2,5.770602,6.623646,6.247059,7.124575,41.0,41.0,41.0,41.0,41.0,41.0,0.419196,0.474619,0.584199,0.461928,1.5,1.5,1.5,1.5,1.5,1.5
3,7.310675,7.840899,6.481517,8.796563,41.0,41.0,41.0,41.0,41.0,41.0,0.563738,0.594554,0.651650,0.605872,1.5,1.5,1.5,1.5,1.5,1.5
4,6.280602,8.135077,6.161432,8.060831,41.0,41.0,41.0,41.0,41.0,41.0,0.464604,0.598588,0.589348,0.535108,1.5,1.5,1.5,1.5,1.5,1.5


In [5]:
#sem_similar_feature.to_csv('data/sem_similar_feature.csv', index=False)
sem_similar_feature = pd.read_csv('data/sem_similar_feature.csv')

## Root Match Feature Added

### example illustration

In [36]:
doc = en_nlp(data.iloc[0,0])
doc

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

In [34]:
# context
data.iloc[0,0]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [35]:
# question
data.iloc[0,1]

'When did Beyonce start becoming popular?'

In [37]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [39]:
[to_nltk_tree(sent.root).pretty_print()  for sent in en_nlp(data.iloc[0,1]).sents]

          start              
  __________|___________      
 |    |     |     |  becoming
 |    |     |     |     |     
When did Beyonce  ?  popular 



[None]

In [40]:
[to_nltk_tree(sent.root) .pretty_print() for sent in doc.sents][0]

                                                                     is                                                                                       
  ___________________________________________________________________|_________________________________________________                                        
 |                                                   Carter                                                          singer                                   
 |      _______________________________________________|_________________________                     _________________|_______________                        
 |     |       |       |     |   |       |        |          |                  born                 |     |      |                songwriter                 
 |     |       |       |     |   |       |        |          |            _______|_______            |     |      |     _______________|_________              
 |     |       |       |     |   |       | 

In [41]:
for sent in doc.sents:
    roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]
    print(roots)

['is', 'is', 'sing', 'songwrit', 'produc']
['in', 'houston', 'perform', 'in', 'as', 'to', 'in', 'as', 'of']
['by', 'fath', 'becam', 'of', 'of']
['saw', 'earn', 'of', 'in', 'est', 'est', 'as', 'earn', 'in', 'singl']


### create root match feature

In [60]:
def match_roots(x):
    """ Find the index of the sentence in the context with matched root with the question"""
    
    question = x["question"].lower()
    sentences = en_nlp(x["context"].lower()).sents
    
    question_root = st.stem(str([sent.root for sent in en_nlp(question).sents][0]))
    
    li = []
    for i,sent in enumerate(sentences):
        roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]

        if question_root in roots: 
            li.append(i)
            
    return li

In [61]:
data["root_match_idx"] = data.apply(match_roots, axis = 1)

In [62]:
def get_columns_from_root(train):
    
    for i in range(len(train)):
        if len(train["root_match_idx"][i]) == 0: pass
        
        else:
            for item in train["root_match_idx"][i]:
                train.loc[i, "column_root_"+"%s"%item] = 1
    return train

In [63]:
data_root = get_columns_from_root(data)
data_root.head()

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,...,column_root_2,column_root_0,column_root_6,column_root_9,column_root_7,column_root_8,column_root_10,column_root_12,column_root_11,column_root_14
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,269,in the late 1990s,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.04716371 0.08111103 ... 0.0...,"[0.6082267463207245, 0.7082076072692871, 0.669...","[8.0352955, 9.462915, 6.914433, 9.608288]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,207,singing and dancing,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.05823731 0.07162362 ... 0.0...,"[0.5250032842159271, 0.51096311211586, 0.62473...","[6.9849253, 6.9543734, 6.4365134, 8.201017]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,526,2003,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,3,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.04716371 0.11232486 ... 0.0...,"[0.4191964268684387, 0.4746186137199402, 0.584...","[5.770602, 6.6236463, 6.2470593, 7.1245747]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,166,"Houston, Texas",['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.03403048 0.08177199 ... 0.0...,"[0.5637381374835968, 0.5945537984371185, 0.651...","[7.3106747, 7.8408995, 6.481517, 8.796563]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,276,late 1990s,['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/...,1,"[array([ 0.00746889, -0.04449936, 0.11347561,...",[[ 0.00746889 -0.06298249 0.10006339 ... 0.0...,"[0.4646044373512268, 0.5985883176326752, 0.589...","[6.2806025, 8.1350765, 6.1614323, 8.060831]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
root_feature = data_root[["column_root_0","column_root_1","column_root_2","column_root_3","column_root_4","column_root_5","column_root_6","column_root_7","column_root_8","column_root_9"]]
root_feature.fillna(0, inplace=True)
root_feature.head()

,column_root_0,column_root_1,column_root_2,column_root_3,column_root_4,column_root_5,column_root_6,column_root_7,column_root_8,column_root_9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#root_feature.to_csv('data/root_features.csv', index=False)
root_feature = pd.read_csv('data/root_features.csv')

## Lemma Match Feature Added

In [223]:
lemmatizer = WordNetLemmatizer() 

In [224]:
def lemma_match(x):
    """ Find the number of common lemma words of each sentence with the question"""
    
    question = x["question"].lower()
    sentences = en_nlp(x["context"].lower()).sents
    
    question_word = word_tokenize(question)
    question_word_lemma = [lemmatizer.lemmatize(word) for word in question_word]  
    
    num_lemma_word = []
    for i, sent in enumerate(sentences):
        sent_word = word_tokenize(str(sent))
        sent_word_lemma = [lemmatizer.lemmatize(word) for word in sent_word]
        
        num = len(list(set(question_word_lemma) & set(sent_word_lemma) - stop_words))
        num_lemma_word.append(num)
        
    diff = 10 - len(num_lemma_word)
    num_lemma_word.extend([None] * diff)
            
    return num_lemma_word

In [225]:
data['num_lemma_word'] = data.apply(lemma_match, axis = 1)

In [226]:
def get_columns_from_lemma_word(train):
    
    columns = ['column_lemma_0', 'column_lemma_1', 'column_lemma_2', 'column_lemma_3', 'column_lemma_4',
              'column_lemma_5', 'column_lemma_6', 'column_lemma_7', 'column_lemma_8', 'column_lemma_9']
    tmp = pd.DataFrame(columns=columns)
    
    for i in range(len(train)):
        tmp.loc[i,] = train['num_lemma_word'][i][:10]

    return tmp

In [232]:
lemma_feature = get_columns_from_lemma_word(data)
lemma_feature.fillna(0, inplace=True)
lemma_feature.head()

,column_lemma_0,column_lemma_1,column_lemma_2,column_lemma_3,column_lemma_4,column_lemma_5,column_lemma_6,column_lemma_7,column_lemma_8,column_lemma_9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,1,4,1,2,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0


In [7]:
#lemma_feature.to_csv('data/lemma_features.csv', index=False)
lemma_feature = pd.read_csv('data/lemma_features.csv')

## Test Different Sets of Features

### Leave lemma match out

In [8]:
data_subset1 = pd.concat([baseline_feature, sem_similar_feature, root_feature], axis=1, join_axes=[baseline_feature.index])
data_subset1.columns

Index(['column_common_0', 'column_common_1', 'column_common_2',
       'column_common_3', 'column_common_4', 'column_common_5',
       'column_common_6', 'column_common_7', 'column_common_8',
       'column_common_9', 'column_euc_0', 'column_euc_1', 'column_euc_2',
       'column_euc_3', 'column_euc_4', 'column_euc_5', 'column_euc_6',
       'column_euc_7', 'column_euc_8', 'column_euc_9', 'column_cos_0',
       'column_cos_1', 'column_cos_2', 'column_cos_3', 'column_cos_4',
       'column_cos_5', 'column_cos_6', 'column_cos_7', 'column_cos_8',
       'column_cos_9', 'column_root_0', 'column_root_1', 'column_root_2',
       'column_root_3', 'column_root_4', 'column_root_5', 'column_root_6',
       'column_root_7', 'column_root_8', 'column_root_9'],
      dtype='object')

In [54]:
train_x, test_x, train_y, test_y = train_test_split(data_subset1, data['target'], train_size=0.7, random_state = 42)
print(train_x.shape)

(57521, 40)


In [55]:
val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, train_size=0.5, random_state = 42)
print(val_x.shape)
print(test_x.shape)

(12326, 40)
(12327, 40)


In [56]:
logistic = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(train_x, train_y)

In [57]:
y_pred = logistic.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.7194775695627484, 0.7194775695627484, 0.7194775695627484, None)
Macro Test Accuracy :  (0.6112114403323986, 0.5672920886630787, 0.5858897076591144, None)
accuracy:  0.7194775695627484


### Leave root match feature out

In [58]:
data_subset2 = pd.concat([baseline_feature, sem_similar_feature, lemma_feature], axis=1, join_axes=[baseline_feature.index])
data_subset2.columns

Index(['column_common_0', 'column_common_1', 'column_common_2',
       'column_common_3', 'column_common_4', 'column_common_5',
       'column_common_6', 'column_common_7', 'column_common_8',
       'column_common_9', 'column_euc_0', 'column_euc_1', 'column_euc_2',
       'column_euc_3', 'column_euc_4', 'column_euc_5', 'column_euc_6',
       'column_euc_7', 'column_euc_8', 'column_euc_9', 'column_cos_0',
       'column_cos_1', 'column_cos_2', 'column_cos_3', 'column_cos_4',
       'column_cos_5', 'column_cos_6', 'column_cos_7', 'column_cos_8',
       'column_cos_9', 'column_lemma_0', 'column_lemma_1', 'column_lemma_2',
       'column_lemma_3', 'column_lemma_4', 'column_lemma_5', 'column_lemma_6',
       'column_lemma_7', 'column_lemma_8', 'column_lemma_9'],
      dtype='object')

In [59]:
train_x, test_x, train_y, test_y = train_test_split(data_subset2, data['target'], train_size=0.7, random_state = 42)
print(train_x.shape)

(57521, 40)


In [60]:
val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, train_size=0.5, random_state = 42)
print(val_x.shape)
print(test_x.shape)

(12326, 40)
(12327, 40)


In [61]:
logistic = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(train_x, train_y)

In [62]:
y_pred = logistic.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.7125010140342338, 0.7125010140342338, 0.7125010140342338, None)
Macro Test Accuracy :  (0.5951361948755083, 0.5562210666856472, 0.5729932738679935, None)
accuracy:  0.7125010140342338


### Leave semantic similarity features out

In [63]:
data_subset3 = pd.concat([baseline_feature, root_feature, lemma_feature], axis=1, join_axes=[baseline_feature.index])
data_subset3.columns

Index(['column_common_0', 'column_common_1', 'column_common_2',
       'column_common_3', 'column_common_4', 'column_common_5',
       'column_common_6', 'column_common_7', 'column_common_8',
       'column_common_9', 'column_root_0', 'column_root_1', 'column_root_2',
       'column_root_3', 'column_root_4', 'column_root_5', 'column_root_6',
       'column_root_7', 'column_root_8', 'column_root_9', 'column_lemma_0',
       'column_lemma_1', 'column_lemma_2', 'column_lemma_3', 'column_lemma_4',
       'column_lemma_5', 'column_lemma_6', 'column_lemma_7', 'column_lemma_8',
       'column_lemma_9'],
      dtype='object')

In [64]:
train_x, test_x, train_y, test_y = train_test_split(data_subset3, data['target'], train_size=0.7, random_state = 42)
print(train_x.shape)

(57521, 30)


In [65]:
val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, train_size=0.5, random_state = 42)
print(val_x.shape)
print(test_x.shape)

(12326, 30)
(12327, 30)


In [66]:
logistic = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(train_x, train_y)

In [67]:
y_pred = logistic.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.6948162569968362, 0.6948162569968362, 0.6948162569968362, None)
Macro Test Accuracy :  (0.5671210886363928, 0.48539593446511947, 0.5158660515684669, None)
accuracy:  0.6948162569968362


### All features used

In [68]:
data_subset = pd.concat([baseline_feature, sem_similar_feature, root_feature, lemma_feature], axis=1, join_axes=[baseline_feature.index])
data_subset.columns

Index(['column_common_0', 'column_common_1', 'column_common_2',
       'column_common_3', 'column_common_4', 'column_common_5',
       'column_common_6', 'column_common_7', 'column_common_8',
       'column_common_9', 'column_euc_0', 'column_euc_1', 'column_euc_2',
       'column_euc_3', 'column_euc_4', 'column_euc_5', 'column_euc_6',
       'column_euc_7', 'column_euc_8', 'column_euc_9', 'column_cos_0',
       'column_cos_1', 'column_cos_2', 'column_cos_3', 'column_cos_4',
       'column_cos_5', 'column_cos_6', 'column_cos_7', 'column_cos_8',
       'column_cos_9', 'column_root_0', 'column_root_1', 'column_root_2',
       'column_root_3', 'column_root_4', 'column_root_5', 'column_root_6',
       'column_root_7', 'column_root_8', 'column_root_9', 'column_lemma_0',
       'column_lemma_1', 'column_lemma_2', 'column_lemma_3', 'column_lemma_4',
       'column_lemma_5', 'column_lemma_6', 'column_lemma_7', 'column_lemma_8',
       'column_lemma_9'],
      dtype='object')

In [69]:
train_x, test_x, train_y, test_y = train_test_split(data_subset, data['target'], train_size=0.7, random_state = 42)
print(train_x.shape)

(57521, 50)


In [70]:
val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, train_size=0.5, random_state = 42)
print(val_x.shape)
print(test_x.shape)

(12326, 50)
(12327, 50)


In [71]:
logistic = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(train_x, train_y)

In [72]:
y_pred = logistic.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.720532165165896, 0.720532165165896, 0.720532165165896, None)
Macro Test Accuracy :  (0.6087193910148803, 0.568055829405579, 0.5856622477845683, None)
accuracy:  0.720532165165896


## Experiment Different Models

In [9]:
train_x, test_x, train_y, test_y = train_test_split(data_subset1, data['target'], train_size=0.7, random_state = 42)
print(train_x.shape)

(57521, 40)


In [10]:
val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, train_size=0.5, random_state = 42)
print(val_x.shape)
print(test_x.shape)

(12326, 40)
(12327, 40)


### Xgboost Model

In [92]:
model = xgb.XGBClassifier()
param_dist = {"max_depth": [3,5,8],
              "learning_rate": [0.05, 0.1, 0.2],
              "n_estimators": [100, 200, 300],
              "colsample_bytree": [0.3, 0.5, 0.7]
               }

# run randomized search
grid_search = GridSearchCV(model, param_grid=param_dist, cv=3, verbose=1, n_jobs=-1)
grid_search.fit(train_x, train_y, 
                eval_set = [(val_x, val_y)],
                early_stopping_rounds=3)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
[0]	validation_0-merror:0.516794
Will train until validation_0-merror hasn't improved in 3 rounds.
[0]	validation_0-merror:0.516226
Will train until validation_0-merror hasn't improved in 3 rounds.
[0]	validation_0-merror:0.517443
Will train until validation_0-merror hasn't improved in 3 rounds.
[0]	validation_0-merror:0.516794
Will train until validation_0-merror hasn't improved in 3 rounds.
[1]	validation_0-merror:0.381957
[1]	validation_0-merror:0.380902
[1]	validation_0-merror:0.381308
[1]	validation_0-merror:0.381957
[2]	validation_0-merror:0.36062
[2]	validation_0-merror:0.362648
[2]	validation_0-merror:0.36062
[2]	validation_0-merror:0.353643
[3]	validation_0-merror:0.341717
[3]	validation_0-merror:0.344151
[3]	validation_0-merror:0.341717
[3]	validation_0-merror:0.346341
[4]	validation_0-merror:0.330927
[4]	validation_0-merror:0.334577
[4]	validation_0-merror:0.330927
[4]	validation_0-merror:0.330602
[5]	validation_0

[16]	validation_0-merror:0.313078
[35]	validation_0-merror:0.295797
[13]	validation_0-merror:0.31754
[17]	validation_0-merror:0.312672
[11]	validation_0-merror:0.294824
[36]	validation_0-merror:0.295311
[14]	validation_0-merror:0.318676
[18]	validation_0-merror:0.31397
[37]	validation_0-merror:0.295473
Stopping. Best iteration:
[34]	validation_0-merror:0.295311

[12]	validation_0-merror:0.294662
[15]	validation_0-merror:0.315269
[19]	validation_0-merror:0.313403
[16]	validation_0-merror:0.314944
[13]	validation_0-merror:0.292552
[20]	validation_0-merror:0.309995
[17]	validation_0-merror:0.315431
[14]	validation_0-merror:0.293039
[21]	validation_0-merror:0.308129
[18]	validation_0-merror:0.315674
[15]	validation_0-merror:0.292309
[22]	validation_0-merror:0.303667
[0]	validation_0-merror:0.481503
Will train until validation_0-merror hasn't improved in 3 rounds.
[19]	validation_0-merror:0.316404
Stopping. Best iteration:
[16]	validation_0-merror:0.314944

[23]	validation_0-merror:0.302775

[13]	validation_0-merror:0.295797
[5]	validation_0-merror:0.306263
[14]	validation_0-merror:0.293039
[14]	validation_0-merror:0.29523
[6]	validation_0-merror:0.300909
[15]	validation_0-merror:0.292309
[7]	validation_0-merror:0.299854
[15]	validation_0-merror:0.294337
[0]	validation_0-merror:0.486695
Will train until validation_0-merror hasn't improved in 3 rounds.
[16]	validation_0-merror:0.289307
[8]	validation_0-merror:0.298799
[16]	validation_0-merror:0.292471
[17]	validation_0-merror:0.290443
[1]	validation_0-merror:0.362242
[9]	validation_0-merror:0.298394
[17]	validation_0-merror:0.29093
[18]	validation_0-merror:0.288658
[2]	validation_0-merror:0.34269
[10]	validation_0-merror:0.300016
[18]	validation_0-merror:0.290605
[19]	validation_0-merror:0.289956
[3]	validation_0-merror:0.31462
[11]	validation_0-merror:0.297663
[19]	validation_0-merror:0.291092
[20]	validation_0-merror:0.288983
[4]	validation_0-merror:0.305371
[20]	validation_0-merror:0.291903
[12]	validation_0-merror:0.296

[17]	validation_0-merror:0.282817
[2]	validation_0-merror:0.325004
[35]	validation_0-merror:0.275515
Stopping. Best iteration:
[32]	validation_0-merror:0.274704

[27]	validation_0-merror:0.277138
[18]	validation_0-merror:0.28014
[3]	validation_0-merror:0.310563
[28]	validation_0-merror:0.276976
[19]	validation_0-merror:0.282573
[4]	validation_0-merror:0.299448
[29]	validation_0-merror:0.276732
[20]	validation_0-merror:0.281519
[5]	validation_0-merror:0.301314
[30]	validation_0-merror:0.27584
[0]	validation_0-merror:0.478014
Will train until validation_0-merror hasn't improved in 3 rounds.
[21]	validation_0-merror:0.279896
[6]	validation_0-merror:0.296852
[31]	validation_0-merror:0.275434
[1]	validation_0-merror:0.348775
[22]	validation_0-merror:0.277949
[7]	validation_0-merror:0.294256
[32]	validation_0-merror:0.275759
[2]	validation_0-merror:0.327113
[23]	validation_0-merror:0.277949
[8]	validation_0-merror:0.292877
[33]	validation_0-merror:0.275515
[3]	validation_0-merror:0.311861
[2

[4]	validation_0-merror:0.323381
[27]	validation_0-merror:0.28809
[43]	validation_0-merror:0.282817
Stopping. Best iteration:
[40]	validation_0-merror:0.282655

[31]	validation_0-merror:0.275434
[5]	validation_0-merror:0.323949
[28]	validation_0-merror:0.286305
[6]	validation_0-merror:0.315106
[32]	validation_0-merror:0.275759
[29]	validation_0-merror:0.286062
[7]	validation_0-merror:0.313727
[30]	validation_0-merror:0.286062
[33]	validation_0-merror:0.275515
[8]	validation_0-merror:0.316323
[31]	validation_0-merror:0.285251
[0]	validation_0-merror:0.516794
Will train until validation_0-merror hasn't improved in 3 rounds.
[9]	validation_0-merror:0.313889
[34]	validation_0-merror:0.275515
Stopping. Best iteration:
[31]	validation_0-merror:0.275434

[32]	validation_0-merror:0.28517
[1]	validation_0-merror:0.376116
[10]	validation_0-merror:0.315836
Stopping. Best iteration:
[7]	validation_0-merror:0.313727

[33]	validation_0-merror:0.284521
[2]	validation_0-merror:0.354292
[34]	validation

...

[38]	validation_0-merror:0.264806
[25]	validation_0-merror:0.269998
[17]	validation_0-merror:0.273
[13]	validation_0-merror:0.273974
[39]	validation_0-merror:0.264319
[26]	validation_0-merror:0.269755
[18]	validation_0-merror:0.272757
Stopping. Best iteration:
[15]	validation_0-merror:0.272351

[14]	validation_0-merror:0.274055
[40]	validation_0-merror:0.263833
[27]	validation_0-merror:0.27081
Stopping. Best iteration:
[24]	validation_0-merror:0.269755

[15]	validation_0-merror:0.273811
[41]	validation_0-merror:0.264238
[16]	validation_0-merror:0.272351
[0]	validation_0-merror:0.41668
Will train until validation_0-merror hasn't improved in 3 rounds.
[42]	validation_0-merror:0.263833
[17]	validation_0-merror:0.271621
[1]	validation_0-merror:0.32971
[0]	validation_0-merror:0.385608
Will train until validation_0-merror hasn't improved in 3 rounds.
[43]	validation_0-merror:0.263427
[18]	validation_0-merror:0.271215
[44]	validation_0-merror:0.263833
[2]	validation_0-merror:0.304722
[1]	vali

[7]	validation_0-merror:0.276002
[1]	validation_0-merror:0.314863
[22]	validation_0-merror:0.269025
Stopping. Best iteration:
[19]	validation_0-merror:0.268863

[2]	validation_0-merror:0.296852
[8]	validation_0-merror:0.274947
[2]	validation_0-merror:0.296933
[3]	validation_0-merror:0.285656
[9]	validation_0-merror:0.275272
[3]	validation_0-merror:0.286873
[4]	validation_0-merror:0.278111
[10]	validation_0-merror:0.273487
[4]	validation_0-merror:0.282249
[5]	validation_0-merror:0.275921
[11]	validation_0-merror:0.273244
[5]	validation_0-merror:0.279572
[6]	validation_0-merror:0.274217
[12]	validation_0-merror:0.273244
[6]	validation_0-merror:0.278111
[7]	validation_0-merror:0.274785
[13]	validation_0-merror:0.272595
[7]	validation_0-merror:0.27584
[8]	validation_0-merror:0.272838
[14]	validation_0-merror:0.272595
[8]	validation_0-merror:0.274704
[9]	validation_0-merror:0.272027
[15]	validation_0-merror:0.271621
[9]	validation_0-merror:0.276408
[10]	validation_0-merror:0.271945
[16]	val

[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 120.6min finished


[0]	validation_0-merror:0.421305
Will train until validation_0-merror hasn't improved in 3 rounds.
[1]	validation_0-merror:0.319406
[2]	validation_0-merror:0.29523
[3]	validation_0-merror:0.286224
[4]	validation_0-merror:0.282168
[5]	validation_0-merror:0.279247
[6]	validation_0-merror:0.278923
[7]	validation_0-merror:0.277706
[8]	validation_0-merror:0.276489
[9]	validation_0-merror:0.275028
[10]	validation_0-merror:0.273487
[11]	validation_0-merror:0.27227
[12]	validation_0-merror:0.270647
[13]	validation_0-merror:0.267727
[14]	validation_0-merror:0.267889
[15]	validation_0-merror:0.267565
[16]	validation_0-merror:0.267646
[17]	validation_0-merror:0.266104
[18]	validation_0-merror:0.266753
[19]	validation_0-merror:0.265699
[20]	validation_0-merror:0.265293
[21]	validation_0-merror:0.265942
[22]	validation_0-merror:0.265374
[23]	validation_0-merror:0.265212
[24]	validation_0-merror:0.265212
[25]	validation_0-merror:0.264482
[26]	validation_0-merror:0.264482
[27]	validation_0-merror:0.2

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 5, 8], 'learning_rate': [0.05, 0.1, 0.2], 'n_estimators': [100, 200, 300], 'colsample_bytree': [0.3, 0.5, 0.7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [93]:
best_param = grid_search.best_estimator_
best_param

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [11]:
best_xgb = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1).fit(train_x, train_y)

In [12]:
y_pred = best_xgb.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.7347286444390363, 0.7347286444390363, 0.7347286444390363, None)
Macro Test Accuracy :  (0.6719842989624266, 0.5602301633533865, 0.600276325728689, None)
accuracy:  0.7347286444390363


### SVM Model

In [81]:
svm = LinearSVC(random_state=42, penalty='l2', C=1).fit(train_x, train_y)

In [82]:
y_pred = svm.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.31491847164760284, 0.31491847164760284, 0.31491847164760284, None)
Macro Test Accuracy :  (0.5847993497626525, 0.3688564892626511, 0.32095578175026684, None)
accuracy:  0.31491847164760284


In [84]:
svm = LinearSVC(random_state=42, penalty='l2', C=0.1).fit(train_x, train_y)

In [85]:
y_pred = svm.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.5909791514561532, 0.5909791514561532, 0.5909791514561532, None)
Macro Test Accuracy :  (0.6498649479479458, 0.44613711920452004, 0.4969057486275758, None)
accuracy:  0.5909791514561532


In [86]:
svm = LinearSVC(random_state=42, penalty='l2', C=2).fit(train_x, train_y)

In [87]:
y_pred = svm.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.6134501500770666, 0.6134501500770666, 0.6134501500770666, None)
Macro Test Accuracy :  (0.605929895377036, 0.48210232287880345, 0.4882672406826828, None)
accuracy:  0.6134501500770666


In [88]:
svm = LinearSVC(random_state=42, penalty='l2', C=0.5).fit(train_x, train_y)

In [89]:
y_pred = svm.predict(test_x)
print("Micro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='micro'))
print("Macro Test Accuracy : ", precision_recall_fscore_support(test_y, y_pred, average='macro'))
print("accuracy: ", np.sum(test_y == y_pred) / len(y_pred))

Micro Test Accuracy :  (0.37803196235904923, 0.37803196235904923, 0.37803196235904923, None)
Macro Test Accuracy :  (0.6459054076187195, 0.3196813753139555, 0.3604502381792756, None)
accuracy:  0.37803196235904923


In [13]:
import joblib
joblib.dump(best_xgb, 'model/xgb_model.sav')

['model/xgb_model.sav']